In [2]:
## Imports
import torch
import torchvision ## Contains some utilities for working with the image data
from torchvision.datasets import MNIST
import matplotlib.pyplot as plt
#%matplotlib inline
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import random_split
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn

In [3]:
mnist_train = datasets.MNIST("./data", train=True, download=True, transform=transforms.ToTensor())
mnist_test = datasets.MNIST("./data", train=False, download=True, transform=transforms.ToTensor())

train_idx = mnist_train.targets <= 1 #only retrieve those with labels less than this value
mnist_train.data = mnist_train.data[train_idx]
mnist_train.targets = mnist_train.targets[train_idx]

test_idx = mnist_test.targets <= 1 #only retrieve those with labels less than this value
mnist_test.data = mnist_test.data[test_idx]
mnist_test.targets = mnist_test.targets[test_idx]

train_loader = DataLoader(mnist_train, batch_size = 100, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size = 100, shuffle=False)


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [6]:
def accuracy(outs, targs):
    _, preds = torch.max(outs, dim = 1)
    return(torch.tensor(torch.sum(preds == targs).item()/ len(preds)))


class linear_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(784, 2)

    def forward (self,inp):
        inp = inp.reshape(-1, 784)
        out = self.linear(inp)
        return(out)

    def train(self, batch):
        imgs, targs = batch
        out = self(imgs)
        loss = F.cross_entropy(out,targs)
        return(loss)

    def validate(self, batch):
        imgs, targs = batch
        out = self(imgs)
        loss = F.cross_entropy(out,targs)
        acc = accuracy(out,targs)
        return({'validate_loss':loss, 'validate_accuracy': acc})

    def validate_epoch_end(self, outs):
        batch_loss = [x['validate_loss'] for x in outs]
        epoch_loss = torch.stack(batch_loss).mean()
        batch_accs = [x['validate_accuracy'] for x in outs]
        epoch_acc = torch.stack(batch_accs).mean()
        return({'validate_loss': epoch_loss.item(), 'validate_accuracy' : epoch_acc.item()})

    def epoch_end(self, epoch,result):
        print("Epoch [{}], validate_loss: {:.4f}, validate_accuracy: {:.4f}".format(epoch, result['validate_loss'], result['validate_accuracy']))

model = linear_model()

In [7]:
def evaluate(model, test_loader):
    outputs = [model.validate(batch) for batch in test_loader]
    return(model.validate_epoch_end(outputs))
    
    
def fit(epochs, lr, model, train_loader, test_loader, opt_func = torch.optim.SGD):
    history = []
    opt = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        
        ## Training Phase
        for batch in train_loader:
            loss = model.train(batch)
            loss.backward()
            opt.step()
            opt.zero_grad()
        
        ## Validation phase
        result = evaluate(model, test_loader)
        model.epoch_end(epoch, result)
        history.append(result)
    return(history)

In [9]:
history1 = fit(5, .01, model, train_loader, test_loader)

Epoch [0], validate_loss: 0.0017, validate_accuracy: 0.9995
Epoch [1], validate_loss: 0.0017, validate_accuracy: 0.9995
Epoch [2], validate_loss: 0.0017, validate_accuracy: 0.9995
Epoch [3], validate_loss: 0.0016, validate_accuracy: 0.9995
Epoch [4], validate_loss: 0.0016, validate_accuracy: 0.9995


In [30]:
class deep_linear_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(784, 20)
        self.linear2 = nn.Linear(20,40)
        self.linear3 = nn.Linear(40,6)

    def forward (self,inp):
        inp = inp.reshape(-1, 784)
        intermed1 = self.linear1(inp)
        intermed1 = F.relu(intermed1)
        intermed2 = self.linear2(intermed1)
        intermed2 = F.relu(intermed2)
        out = self.linear3(intermed2)
        return(out)

    def train(self, batch):
        imgs, targs = batch
        out = self(imgs)
        loss = F.cross_entropy(out,targs)
        return(loss)

    def validate(self, batch):
        imgs, targs = batch
        out = self(imgs)
        loss = F.cross_entropy(out,targs)
        acc = accuracy(out,targs)
        return({'validate_loss':loss, 'validate_accuracy': acc})

    def validate_epoch_end(self, outs):
          batch_loss = [x['validate_loss'] for x in outs]
          epoch_loss = torch.stack(batch_loss).mean()
          batch_accs = [x['validate_accuracy'] for x in outs]
          epoch_acc = torch.stack(batch_accs).mean()
          return({'validate_loss': epoch_loss.item(), 'validate_accuracy' : epoch_acc.item()})

    def epoch_end(self, epoch,result):
          print("Epoch [{}], validate_loss: {:.4f}, validate_accuracy: {:.4f}".format(epoch, result['validate_loss'], result['validate_accuracy']))

model2 = deep_linear_model()

In [48]:
history2 = fit(10, 0.01, model2, train_loader, test_loader)

Epoch [0], validate_loss: 0.3983, validate_accuracy: 0.8961
Epoch [1], validate_loss: 0.2609, validate_accuracy: 0.9271
Epoch [2], validate_loss: 0.2125, validate_accuracy: 0.9392
Epoch [3], validate_loss: 0.1860, validate_accuracy: 0.9437
Epoch [4], validate_loss: 0.1751, validate_accuracy: 0.9497
Epoch [5], validate_loss: 0.1645, validate_accuracy: 0.9497
Epoch [6], validate_loss: 0.1580, validate_accuracy: 0.9551
Epoch [7], validate_loss: 0.1506, validate_accuracy: 0.9561
Epoch [8], validate_loss: 0.1442, validate_accuracy: 0.9573
Epoch [9], validate_loss: 0.1424, validate_accuracy: 0.9584


In [49]:
class deep_conv_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,8,3)
        self.conv2 = nn.Conv2d(8,1,3)
        self.linear1 = nn.Linear(8*8*9,6)

    def forward (self,inp):
#    inp = inp.reshape(-1, 784)
        intermed1 = self.conv1(inp)
        intermed1 = F.relu(intermed1)
        intermed2 = self.conv2(intermed1)
        intermed2 = F.relu(intermed2)
        intermed2 = intermed2.view(intermed2.size(0), -1) 
        out = self.linear1(intermed2)
        return(out)

    def train(self, batch):
        imgs, targs = batch
        out = self(imgs)
        loss = F.cross_entropy(out,targs)
        return(loss)

    def validate(self, batch):
        imgs, targs = batch
        out = self(imgs)
        loss = F.cross_entropy(out,targs)
        acc = accuracy(out,targs)
        return({'validate_loss':loss, 'validate_accuracy': acc})

    def validate_epoch_end(self, outs):
        batch_loss = [x['validate_loss'] for x in outs]
        epoch_loss = torch.stack(batch_loss).mean()
        batch_accs = [x['validate_accuracy'] for x in outs]
        epoch_acc = torch.stack(batch_accs).mean()
        return({'validate_loss': epoch_loss.item(), 'validate_accuracy' : epoch_acc.item()})

    def epoch_end(self, epoch,result):
        print("Epoch [{}], validate_loss: {:.4f}, validate_accuracy: {:.4f}".format(epoch, result['validate_loss'], result['validate_accuracy']))

model3 = deep_conv_model()

In [50]:
history3 = fit(10, 0.001, model3, train_loader, test_loader)

Epoch [0], validate_loss: 1.7619, validate_accuracy: 0.2574
Epoch [1], validate_loss: 1.6853, validate_accuracy: 0.5791
Epoch [2], validate_loss: 1.3654, validate_accuracy: 0.7409
Epoch [3], validate_loss: 0.6109, validate_accuracy: 0.8521
Epoch [4], validate_loss: 0.3524, validate_accuracy: 0.8989
Epoch [5], validate_loss: 0.2704, validate_accuracy: 0.9235
Epoch [6], validate_loss: 0.2323, validate_accuracy: 0.9342
Epoch [7], validate_loss: 0.2136, validate_accuracy: 0.9401
Epoch [8], validate_loss: 0.1979, validate_accuracy: 0.9414
Epoch [9], validate_loss: 0.1896, validate_accuracy: 0.9440
